# Análisis y Limpieza de Datos - Tabla Productos

Este notebook contiene el análisis completo de la tabla **Productos** para identificar problemas de calidad de datos, realizar limpieza, estandarización y normalización.

## Objetivos:
1. **Análisis de problemas**: Identificar duplicados, valores nulos, formatos incorrectos
2. **Limpieza**: Eliminar espacios extra, corregir formatos
3. **Estandarización**: Unificar formatos de precios, nombres y categorías
4. **Normalización**: Verificar relaciones con otras tablas

## Herramientas utilizadas:
- **Pandas**: Para manipulación y análisis de datos
- **Numpy**: Para operaciones numéricas eficientes

In [25]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re

# Configuración para mostrar más columnas y filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Librerías importadas correctamente ✓")

Librerías importadas correctamente ✓


## 1. Carga de Datos

Cargamos la tabla **Productos** y las tablas relacionadas para entender la estructura completa de los datos.

In [26]:
# Definir rutas de archivos
ruta_base = '../Base_de_datos/'
ruta_limpia = '../Base_de_datos_limpia/'

# Crear carpeta de destino si no existe
os.makedirs(ruta_limpia, exist_ok=True)

# Cargar tabla principal Productos
df_productos = pd.read_excel(ruta_base + 'Productos.xlsx')

# Cargar tablas relacionales para verificar integridad
df_detalle_ventas = pd.read_excel(ruta_base + 'Detalle_ventas.xlsx')
df_clientes = pd.read_excel(ruta_base + 'Clientes.xlsx')
df_ventas = pd.read_excel(ruta_base + 'Ventas.xlsx')

print("✓ Datos cargados correctamente")
print(f"Productos: {df_productos.shape[0]} filas, {df_productos.shape[1]} columnas")
print(f"Detalle Ventas: {df_detalle_ventas.shape[0]} filas")
print(f"Clientes: {df_clientes.shape[0]} filas")
print(f"Ventas: {df_ventas.shape[0]} filas")

✓ Datos cargados correctamente
Productos: 100 filas, 4 columnas
Detalle Ventas: 343 filas
Clientes: 100 filas
Ventas: 120 filas


## 2. Exploración Inicial

Examinamos la estructura y características básicas de la tabla Productos.

In [27]:
# Exploración inicial de la tabla Productos
print("=== INFORMACIÓN GENERAL ===")
print(f"Dimensiones: {df_productos.shape}")
print(f"Columnas: {list(df_productos.columns)}")
print("\n=== TIPOS DE DATOS ===")
print(df_productos.dtypes)
print("\n=== PRIMERAS 3 FILAS ===")
print(df_productos.head(3))
print("\n=== INFORMACIÓN ESTADÍSTICA ===")
print(df_productos.describe(include='all'))

=== INFORMACIÓN GENERAL ===
Dimensiones: (100, 4)
Columnas: ['id_producto', 'nombre_producto', 'categoria', 'precio_unitario']

=== TIPOS DE DATOS ===
id_producto         int64
nombre_producto    object
categoria          object
precio_unitario     int64
dtype: object

=== PRIMERAS 3 FILAS ===
   id_producto nombre_producto  categoria  precio_unitario
0            1  Coca Cola 1.5L  Alimentos             2347
1            2      Pepsi 1.5L   Limpieza             4973
2            3     Sprite 1.5L  Alimentos             4964

=== INFORMACIÓN ESTADÍSTICA ===
        id_producto nombre_producto  categoria  precio_unitario
count    100.000000             100        100       100.000000
unique          NaN             100          2              NaN
top             NaN  Coca Cola 1.5L  Alimentos              NaN
freq            NaN               1         50              NaN
mean      50.500000             NaN        NaN      2718.550000
std       29.011492             NaN        NaN      

## 3. Análisis de Problemas

Identificamos problemas específicos en cada columna: duplicados, valores nulos, formatos incorrectos en precios y categorías.

In [28]:
# Análisis detallado de problemas en los datos
print("=== ANÁLISIS DE PROBLEMAS ===\n")

# 1. Verificar valores nulos en cada columna
print("1. VALORES NULOS POR COLUMNA:")
valores_nulos = df_productos.isnull().sum()
print(valores_nulos[valores_nulos > 0])  # Solo mostrar columnas con nulos
if valores_nulos.sum() == 0:
    print("✓ No hay valores nulos")

# 2. Verificar duplicados en ID_producto (si existe)
if 'id_producto' in df_productos.columns:
    print("\n2. ANÁLISIS DE ID_PRODUCTO:")
    duplicados_id = df_productos['id_producto'].duplicated().sum()
    print(f"IDs duplicados: {duplicados_id}")
    if duplicados_id > 0:
        print("IDs duplicados encontrados:")
        print(df_productos[df_productos['id_producto'].duplicated(keep=False)]['id_producto'].values)
else:
    print("\n2. ANÁLISIS DE ID_PRODUCTO:")
    print("No se encontró columna 'id_producto'")

# 3. Verificar filas completamente duplicadas
print("\n3. FILAS DUPLICADAS COMPLETAS:")
filas_duplicadas = df_productos.duplicated().sum()
print(f"Filas duplicadas: {filas_duplicadas}")

# 4. Verificar problemas en nombres de productos (espacios extra)
if 'nombre_producto' in df_productos.columns:
    print("\n4. PROBLEMAS EN NOMBRE_PRODUCTO:")
    nombres_con_espacios = df_productos['nombre_producto'].str.strip() != df_productos['nombre_producto']
    print(f"Nombres con espacios al inicio/final: {nombres_con_espacios.sum()}")
else:
    print("\n4. ANÁLISIS DE NOMBRES:")
    # Buscar columnas que puedan ser nombres
    posibles_nombres = [col for col in df_productos.columns if 'nombre' in col.lower() or 'producto' in col.lower()]
    if posibles_nombres:
        print(f"Columnas encontradas: {posibles_nombres}")
        for col in posibles_nombres:
            espacios = df_productos[col].str.strip() != df_productos[col]
            print(f"{col} - espacios extra: {espacios.sum()}")
    else:
        print("No se encontraron columnas de nombre de producto")

=== ANÁLISIS DE PROBLEMAS ===

1. VALORES NULOS POR COLUMNA:
Series([], dtype: int64)
✓ No hay valores nulos

2. ANÁLISIS DE ID_PRODUCTO:
IDs duplicados: 0

3. FILAS DUPLICADAS COMPLETAS:
Filas duplicadas: 0

4. PROBLEMAS EN NOMBRE_PRODUCTO:
Nombres con espacios al inicio/final: 0


In [29]:
# Continuación del análisis de problemas - Precios y categorías
print("5. PROBLEMAS EN PRECIOS:")
# Buscar columnas de precios
columnas_precio = [col for col in df_productos.columns if 'precio' in col.lower() or 'cost' in col.lower() or 'valor' in col.lower()]
if columnas_precio:
    for col in columnas_precio:
        print(f"\n   Análisis de {col}:")
        # Verificar si es numérico
        try:
            valores_numericos = pd.to_numeric(df_productos[col], errors='coerce')
            valores_no_numericos = valores_numericos.isnull().sum() - df_productos[col].isnull().sum()
            print(f"   - Valores no numéricos: {valores_no_numericos}")
            
            # Verificar precios negativos o cero
            if not valores_numericos.empty:
                precios_negativos = (valores_numericos < 0).sum()
                precios_cero = (valores_numericos == 0).sum()
                print(f"   - Precios negativos: {precios_negativos}")
                print(f"   - Precios en cero: {precios_cero}")
                
                # Mostrar estadísticas básicas
                print(f"   - Rango: ${valores_numericos.min():.2f} - ${valores_numericos.max():.2f}")
        except Exception as e:
            print(f"   - Error al analizar {col}: {e}")
else:
    print("No se encontraron columnas de precios")

print("\n6. PROBLEMAS EN CATEGORÍAS:")
# Buscar columnas de categoría
columnas_categoria = [col for col in df_productos.columns if 'categoria' in col.lower() or 'tipo' in col.lower()]
if columnas_categoria:
    for col in columnas_categoria:
        print(f"\n   Análisis de {col}:")
        categorias_con_espacios = df_productos[col].str.strip() != df_productos[col]
        print(f"   - Categorías con espacios extra: {categorias_con_espacios.sum()}")
        print(f"   - Categorías únicas: {df_productos[col].nunique()}")
        print(f"   - Primeras 5 categorías: {list(df_productos[col].unique()[:5])}")
else:
    print("No se encontraron columnas de categoría")

print("\n7. PROBLEMAS EN DESCRIPCIONES:")
# Buscar columnas de descripción
columnas_descripcion = [col for col in df_productos.columns if 'descripcion' in col.lower() or 'desc' in col.lower()]
if columnas_descripcion:
    for col in columnas_descripcion:
        print(f"\n   Análisis de {col}:")
        descripciones_vacias = df_productos[col].str.strip().eq('').sum()
        print(f"   - Descripciones vacías: {descripciones_vacias}")
        long_promedio = df_productos[col].str.len().mean()
        print(f"   - Longitud promedio: {long_promedio:.1f} caracteres")
else:
    print("No se encontraron columnas de descripción")

5. PROBLEMAS EN PRECIOS:

   Análisis de precio_unitario:
   - Valores no numéricos: 0
   - Precios negativos: 0
   - Precios en cero: 0
   - Rango: $272.00 - $4982.00

6. PROBLEMAS EN CATEGORÍAS:

   Análisis de categoria:
   - Categorías con espacios extra: 0
   - Categorías únicas: 2
   - Primeras 5 categorías: ['Alimentos', 'Limpieza']

7. PROBLEMAS EN DESCRIPCIONES:
No se encontraron columnas de descripción


## 4. Verificación de Integridad Relacional

Verificamos que los IDs de productos en la tabla **Productos** coincidan con los de las tablas relacionadas (**Detalle_ventas**).

In [30]:
# Verificación de integridad relacional
print("=== VERIFICACIÓN DE INTEGRIDAD RELACIONAL ===\n")

# Buscar columna de ID en productos
columna_id_producto = None
for col in df_productos.columns:
    if 'id' in col.lower() and 'producto' in col.lower():
        columna_id_producto = col
        break

if columna_id_producto:
    # Obtener IDs únicos de productos
    ids_productos = set(df_productos[columna_id_producto].unique())
    print(f"IDs únicos en Productos: {len(ids_productos)}")
    
    # Buscar columna correspondiente en detalle_ventas
    columna_id_detalle = None
    for col in df_detalle_ventas.columns:
        if 'id' in col.lower() and 'producto' in col.lower():
            columna_id_detalle = col
            break
    
    if columna_id_detalle:
        ids_detalle = set(df_detalle_ventas[columna_id_detalle].unique())
        print(f"IDs únicos en Detalle_ventas: {len(ids_detalle)}")
        
        # Verificar productos sin ventas (huérfanos)
        productos_sin_ventas = ids_productos - ids_detalle
        print(f"Productos sin ventas: {len(productos_sin_ventas)}")
        
        # Verificar ventas de productos inexistentes
        ventas_sin_producto = ids_detalle - ids_productos
        print(f"Ventas con productos inexistentes: {len(ventas_sin_producto)}")
        
        if ventas_sin_producto:
            print(f"IDs problemáticos en detalle_ventas: {list(ventas_sin_producto)[:5]}...")
    else:
        print("No se encontró columna de ID de producto en Detalle_ventas")
        print(f"Columnas disponibles: {list(df_detalle_ventas.columns)}")
else:
    print("No se encontró columna de ID de producto")
    print(f"Columnas disponibles en Productos: {list(df_productos.columns)}")

# Verificar nombres duplicados (productos con mismo nombre pero diferente ID)
nombre_cols = [col for col in df_productos.columns if 'nombre' in col.lower()]
if nombre_cols:
    col_nombre = nombre_cols[0]
    print(f"\n=== NOMBRES DUPLICADOS ===")
    nombres_duplicados = df_productos[col_nombre].duplicated().sum()
    print(f"Nombres de productos duplicados: {nombres_duplicados}")
    if nombres_duplicados > 0:
        print("Productos con nombres duplicados:")
        productos_repetidos = df_productos[df_productos[col_nombre].duplicated(keep=False)]
        if columna_id_producto:
            print(productos_repetidos[[columna_id_producto, col_nombre]].head())
        else:
            print(productos_repetidos[col_nombre].head())

=== VERIFICACIÓN DE INTEGRIDAD RELACIONAL ===

IDs únicos en Productos: 100
IDs únicos en Detalle_ventas: 95
Productos sin ventas: 5
Ventas con productos inexistentes: 0

=== NOMBRES DUPLICADOS ===
Nombres de productos duplicados: 0
IDs únicos en Productos: 100
IDs únicos en Detalle_ventas: 95
Productos sin ventas: 5
Ventas con productos inexistentes: 0

=== NOMBRES DUPLICADOS ===
Nombres de productos duplicados: 0


## 5. Limpieza y Estandarización

Aplicamos correcciones a los problemas identificados usando operaciones eficientes de **pandas**.

In [31]:
# Crear copia para limpieza (preservar datos originales)
df_productos_limpio = df_productos.copy()

print("=== PROCESO DE LIMPIEZA ===\n")

# 1. Limpiar espacios en blanco en todas las columnas de texto
print("1. Eliminando espacios extra...")
columnas_texto = df_productos_limpio.select_dtypes(include=['object']).columns
for col in columnas_texto:
    df_productos_limpio[col] = df_productos_limpio[col].astype(str).str.strip()

# 2. Estandarizar nombres de productos (formato título si existe)
nombre_cols = [col for col in df_productos_limpio.columns if 'nombre' in col.lower()]
if nombre_cols:
    print("2. Estandarizando nombres de productos...")
    for col in nombre_cols:
        df_productos_limpio[col] = df_productos_limpio[col].str.title()
else:
    print("2. No se encontraron columnas de nombre para estandarizar")

# 3. Estandarizar categorías (formato título si existe)
categoria_cols = [col for col in df_productos_limpio.columns if 'categoria' in col.lower() or 'tipo' in col.lower()]
if categoria_cols:
    print("3. Estandarizando categorías...")
    for col in categoria_cols:
        df_productos_limpio[col] = df_productos_limpio[col].str.title()
else:
    print("3. No se encontraron columnas de categoría para estandarizar")

# 4. Convertir precios a formato numérico
precio_cols = [col for col in df_productos_limpio.columns if 'precio' in col.lower() or 'cost' in col.lower() or 'valor' in col.lower()]
if precio_cols:
    print("4. Estandarizando precios...")
    for col in precio_cols:
        try:
            # Remover símbolos de moneda y espacios
            df_productos_limpio[col] = df_productos_limpio[col].astype(str).str.replace('[$,€]', '', regex=True)
            df_productos_limpio[col] = pd.to_numeric(df_productos_limpio[col], errors='coerce')
            print(f"   - {col}: convertido a numérico")
        except Exception as e:
            print(f"   - Error en {col}: {e}")
else:
    print("4. No se encontraron columnas de precio para estandarizar")

# 5. Eliminar filas duplicadas completas si existen
filas_antes = len(df_productos_limpio)
df_productos_limpio = df_productos_limpio.drop_duplicates()
filas_despues = len(df_productos_limpio)
print(f"5. Filas duplicadas eliminadas: {filas_antes - filas_despues}")

print(f"\n✓ Limpieza completada. Registros procesados: {len(df_productos_limpio)}")

=== PROCESO DE LIMPIEZA ===

1. Eliminando espacios extra...
2. Estandarizando nombres de productos...
3. Estandarizando categorías...
4. Estandarizando precios...
   - precio_unitario: convertido a numérico
5. Filas duplicadas eliminadas: 0

✓ Limpieza completada. Registros procesados: 100
4. Estandarizando precios...
   - precio_unitario: convertido a numérico
5. Filas duplicadas eliminadas: 0

✓ Limpieza completada. Registros procesados: 100


## 6. Validación de Datos Limpios

Verificamos que las correcciones se aplicaron correctamente y identificamos registros que requieren atención manual.

In [32]:
# Validación post-limpieza
print("=== VALIDACIÓN POST-LIMPIEZA ===\n")

# 1. Verificar precios después de la limpieza
precio_cols = [col for col in df_productos_limpio.columns if 'precio' in col.lower()]
if precio_cols:
    print("1. VALIDACIÓN DE PRECIOS:")
    for col in precio_cols:
        precios_nulos = df_productos_limpio[col].isnull().sum()
        precios_negativos = (df_productos_limpio[col] < 0).sum()
        precios_cero = (df_productos_limpio[col] == 0).sum()
        print(f"   {col}:")
        print(f"   - Valores nulos: {precios_nulos}")
        print(f"   - Valores negativos: {precios_negativos}")
        print(f"   - Valores en cero: {precios_cero}")
else:
    print("1. No hay columnas de precio para validar")

# 2. Verificar IDs duplicados
id_cols = [col for col in df_productos_limpio.columns if 'id' in col.lower()]
if id_cols:
    print(f"\n2. VALIDACIÓN DE IDs:")
    for col in id_cols:
        ids_duplicados = df_productos_limpio[col].duplicated().sum()
        print(f"   {col} - IDs duplicados: {ids_duplicados}")
else:
    print("\n2. No hay columnas de ID para validar")

# 3. Verificar nombres duplicados
nombre_cols = [col for col in df_productos_limpio.columns if 'nombre' in col.lower()]
if nombre_cols:
    print(f"\n3. VALIDACIÓN DE NOMBRES:")
    for col in nombre_cols:
        nombres_duplicados = df_productos_limpio[col].duplicated().sum()
        print(f"   {col} - Nombres duplicados: {nombres_duplicados}")
else:
    print("\n3. No hay columnas de nombre para validar")

# 4. Identificar registros problemáticos
print(f"\n=== REGISTROS QUE REQUIEREN ATENCIÓN MANUAL ===")

problemas_encontrados = []

# Buscar registros con precios problemáticos
for col in precio_cols:
    mask_problemas = (df_productos_limpio[col].isnull()) | (df_productos_limpio[col] < 0)
    if mask_problemas.any():
        problemas_encontrados.extend(df_productos_limpio[mask_problemas].index.tolist())

# Buscar registros con IDs duplicados
for col in id_cols:
    mask_duplicados = df_productos_limpio[col].duplicated(keep=False)
    if mask_duplicados.any():
        problemas_encontrados.extend(df_productos_limpio[mask_duplicados].index.tolist())

# Remover duplicados en la lista de problemas
problemas_unicos = list(set(problemas_encontrados))

print(f"Total de registros con problemas: {len(problemas_unicos)}")

if len(problemas_unicos) > 0:
    print(f"\nPrimeros 5 registros problemáticos:")
    print(df_productos_limpio.iloc[problemas_unicos[:5]])
else:
    print("✓ Todos los registros están limpios")

=== VALIDACIÓN POST-LIMPIEZA ===

1. VALIDACIÓN DE PRECIOS:
   precio_unitario:
   - Valores nulos: 0
   - Valores negativos: 0
   - Valores en cero: 0

2. VALIDACIÓN DE IDs:
   id_producto - IDs duplicados: 0

3. VALIDACIÓN DE NOMBRES:
   nombre_producto - Nombres duplicados: 0

=== REGISTROS QUE REQUIEREN ATENCIÓN MANUAL ===
Total de registros con problemas: 0
✓ Todos los registros están limpios


## 7. Normalización Final

Aplicamos las últimas optimizaciones y creamos el dataset final normalizado.

In [33]:
# Normalización final y optimizaciones
print("=== NORMALIZACIÓN FINAL ===\n")

# 1. Ordenar por ID para consistencia (si existe columna de ID)
id_cols = [col for col in df_productos_limpio.columns if 'id' in col.lower()]
if id_cols:
    col_id = id_cols[0]
    df_productos_final = df_productos_limpio.sort_values(col_id).reset_index(drop=True)
    print(f"1. Datos ordenados por {col_id}")
else:
    df_productos_final = df_productos_limpio.reset_index(drop=True)
    print("1. Datos ordenados por índice (no se encontró columna ID)")

# 2. Optimizar tipos de datos para eficiencia
print("2. Optimizando tipos de datos...")

# Convertir IDs a entero si es posible
for col in id_cols:
    try:
        df_productos_final[col] = pd.to_numeric(df_productos_final[col], downcast='integer')
        print(f"   - {col}: convertido a entero")
    except:
        print(f"   - {col}: mantiene formato original")

# Categorizar columnas con pocas categorías únicas
categoria_cols = [col for col in df_productos_final.columns if 'categoria' in col.lower() or 'tipo' in col.lower()]
for col in categoria_cols:
    categorias_unicas = df_productos_final[col].nunique()
    total_registros = len(df_productos_final)
    
    if categorias_unicas < total_registros * 0.5:  # Si hay menos del 50% de categorías únicas
        df_productos_final[col] = df_productos_final[col].astype('category')
        print(f"   - {col}: convertido a categoría ({categorias_unicas} categorías únicas)")

# 3. Crear columnas adicionales útiles para análisis
print("3. Creando columnas derivadas...")

# Calcular rango de precios si hay columnas de precio
precio_cols = [col for col in df_productos_final.columns if 'precio' in col.lower()]
if len(precio_cols) >= 1:
    col_precio = precio_cols[0]
    # Crear categorías de precio
    try:
        df_productos_final['rango_precio'] = pd.cut(
            df_productos_final[col_precio], 
            bins=5, 
            labels=['Muy Bajo', 'Bajo', 'Medio', 'Alto', 'Muy Alto']
        )
        print(f"   - rango_precio: creado basado en {col_precio}")
    except:
        print(f"   - No se pudo crear rango_precio desde {col_precio}")

# Crear indicador de producto activo si hay columnas de descripción
desc_cols = [col for col in df_productos_final.columns if 'descripcion' in col.lower()]
if desc_cols:
    col_desc = desc_cols[0]
    df_productos_final['tiene_descripcion'] = ~df_productos_final[col_desc].isnull()
    print(f"   - tiene_descripcion: indicador creado")

print(f"\n✓ Normalización completada")
print(f"✓ Dataset final: {len(df_productos_final)} registros")
print(f"✓ Columnas finales: {list(df_productos_final.columns)}")

=== NORMALIZACIÓN FINAL ===

1. Datos ordenados por id_producto
2. Optimizando tipos de datos...
   - id_producto: convertido a entero
   - categoria: convertido a categoría (2 categorías únicas)
3. Creando columnas derivadas...
   - rango_precio: creado basado en precio_unitario

✓ Normalización completada
✓ Dataset final: 100 registros
✓ Columnas finales: ['id_producto', 'nombre_producto', 'categoria', 'precio_unitario', 'rango_precio']


### 📊 **Técnica Avanzada: One-Hot Encoding**

Cuando tenemos **pocas categorías únicas** (2-10), es recomendable aplicar **One-Hot Encoding** para facilitar análisis y machine learning.

In [34]:
# One-Hot Encoding para categorías con pocas opciones únicas
print("4. Aplicando One-Hot Encoding...")

categoria_cols_para_encoding = []

# Buscar columnas categóricas con pocas categorías únicas (ideal para One-Hot)
for col in categoria_cols:
    categorias_unicas = df_productos_final[col].nunique()
    
    # Aplicar One-Hot solo si hay entre 2 y 10 categorías
    if 2 <= categorias_unicas <= 10:
        categoria_cols_para_encoding.append(col)
        
        print(f"   - {col}: {categorias_unicas} categorías → Aplicando One-Hot Encoding")
        
        # Crear columnas dummy (One-Hot)
        dummies = pd.get_dummies(df_productos_final[col], prefix=col, dtype=int)
        
        # Agregar las nuevas columnas al DataFrame
        df_productos_final = pd.concat([df_productos_final, dummies], axis=1)
        
        # Opcional: Eliminar la columna original (recomendado para análisis)
        # df_productos_final = df_productos_final.drop(col, axis=1)
        
        print(f"      ✓ Creadas {len(dummies.columns)} columnas binarias: {list(dummies.columns)}")
        
    elif categorias_unicas > 10:
        print(f"   - {col}: {categorias_unicas} categorías → NO recomendado para One-Hot (demasiadas)")
    else:
        print(f"   - {col}: {categorias_unicas} categoría → NO necesario One-Hot (solo una opción)")

if not categoria_cols_para_encoding:
    print("   - No se encontraron columnas apropiadas para One-Hot Encoding")
else:
    print(f"\\n✓ One-Hot Encoding aplicado a {len(categoria_cols_para_encoding)} columnas")
    print(f"✓ Columnas totales ahora: {len(df_productos_final.columns)}")

# Mostrar ejemplo de las nuevas columnas
if categoria_cols_para_encoding:
    print(f"\\n📋 EJEMPLO DE ONE-HOT ENCODING:")
    # Mostrar original + nuevas columnas para comparar
    col_ejemplo = categoria_cols_para_encoding[0]
    columnas_nuevas = [col for col in df_productos_final.columns if col.startswith(col_ejemplo + '_')]
    columnas_mostrar = [col_ejemplo] + columnas_nuevas
    print(df_productos_final[columnas_mostrar].head(3))

4. Aplicando One-Hot Encoding...
   - categoria: 2 categorías → Aplicando One-Hot Encoding
      ✓ Creadas 2 columnas binarias: ['categoria_Alimentos', 'categoria_Limpieza']
\n✓ One-Hot Encoding aplicado a 1 columnas
✓ Columnas totales ahora: 7
\n📋 EJEMPLO DE ONE-HOT ENCODING:
   categoria  categoria_Alimentos  categoria_Limpieza
0  Alimentos                    1                   0
1   Limpieza                    0                   1
2  Alimentos                    1                   0


## 8. Reporte Final y Guardado

Generamos un reporte de todas las transformaciones realizadas y guardamos los datos limpios.

In [35]:
# Reporte final de transformaciones
print("=== REPORTE FINAL DE LIMPIEZA ===\n")

# Comparación antes vs después
print("RESUMEN DE TRANSFORMACIONES:")
print(f"📊 Registros originales: {len(df_productos)}")
print(f"📊 Registros finales: {len(df_productos_final)}")
print(f"📊 Registros eliminados: {len(df_productos) - len(df_productos_final)}")

print(f"\n📋 CAMBIOS APLICADOS:")
print("   ✓ Eliminación de espacios extra en textos")
print("   ✓ Estandarización de nombres de productos")
print("   ✓ Estandarización de categorías")
print("   ✓ Conversión de precios a formato numérico")
print("   ✓ Eliminación de filas duplicadas")
print("   ✓ Optimización de tipos de datos")
print("   ✓ Creación de columnas derivadas")

print(f"\n📈 CALIDAD DE DATOS:")

# Contar registros válidos en columnas clave
precio_cols = [col for col in df_productos_final.columns if 'precio' in col.lower()]
if precio_cols:
    precios_validos = df_productos_final[precio_cols[0]].notna().sum()
    print(f"   ✓ Precios válidos: {precios_validos}/{len(df_productos_final)} ({precios_validos/len(df_productos_final)*100:.1f}%)")

id_cols = [col for col in df_productos_final.columns if 'id' in col.lower()]
if id_cols:
    ids_unicos = df_productos_final[id_cols[0]].nunique() == len(df_productos_final)
    print(f"   ✓ IDs únicos: {ids_unicos}")

categoria_cols = [col for col in df_productos_final.columns if 'categoria' in col.lower()]
if categoria_cols:
    categorias_count = df_productos_final[categoria_cols[0]].nunique()
    print(f"   ✓ Categorías únicas: {categorias_count}")

# Mostrar muestra de datos finales
print(f"\n📋 MUESTRA DE DATOS LIMPIOS:")
print(df_productos_final.head(3))

=== REPORTE FINAL DE LIMPIEZA ===

RESUMEN DE TRANSFORMACIONES:
📊 Registros originales: 100
📊 Registros finales: 100
📊 Registros eliminados: 0

📋 CAMBIOS APLICADOS:
   ✓ Eliminación de espacios extra en textos
   ✓ Estandarización de nombres de productos
   ✓ Estandarización de categorías
   ✓ Conversión de precios a formato numérico
   ✓ Eliminación de filas duplicadas
   ✓ Optimización de tipos de datos
   ✓ Creación de columnas derivadas

📈 CALIDAD DE DATOS:
   ✓ Precios válidos: 100/100 (100.0%)
   ✓ IDs únicos: True
   ✓ Categorías únicas: 2

📋 MUESTRA DE DATOS LIMPIOS:
   id_producto nombre_producto  categoria  precio_unitario rango_precio  \
0            1  Coca Cola 1.5L  Alimentos             2347        Medio   
1            2      Pepsi 1.5L   Limpieza             4973     Muy Alto   
2            3     Sprite 1.5L  Alimentos             4964     Muy Alto   

   categoria_Alimentos  categoria_Limpieza  
0                    1                   0  
1                    0     

In [36]:
# Guardar datos limpios en la carpeta destino
archivo_salida = ruta_limpia + 'Productos_limpio.xlsx'

# Guardar en Excel
df_productos_final.to_excel(archivo_salida, index=False)

# Guardar también en CSV para compatibilidad
archivo_csv = ruta_limpia + 'Productos_limpio.csv'
df_productos_final.to_csv(archivo_csv, index=False, encoding='utf-8')

print("=== ARCHIVOS GUARDADOS ===")
print(f"✅ Excel: {archivo_salida}")
print(f"✅ CSV: {archivo_csv}")

# Crear reporte de limpieza
reporte = f"""
REPORTE DE LIMPIEZA - TABLA PRODUCTOS
=====================================
Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

RESUMEN:
- Registros originales: {len(df_productos)}
- Registros finales: {len(df_productos_final)}
- Registros eliminados: {len(df_productos) - len(df_productos_final)}

TRANSFORMACIONES APLICADAS:
1. Eliminación de espacios extra
2. Estandarización de nombres de productos
3. Estandarización de categorías
4. Conversión de precios a formato numérico
5. Eliminación de duplicados
6. Optimización de tipos de datos
7. Creación de columnas derivadas

COLUMNAS FINALES:
{list(df_productos_final.columns)}

CALIDAD FINAL:
- Total de productos: {len(df_productos_final)}
- Columnas procesadas: {len(df_productos_final.columns)}
"""

# Guardar reporte
with open(ruta_limpia + 'Reporte_Limpieza_Productos.txt', 'w', encoding='utf-8') as f:
    f.write(reporte)

print(f"✅ Reporte: {ruta_limpia}Reporte_Limpieza_Productos.txt")
print(f"\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!")
print(f"📁 Archivos disponibles en: {ruta_limpia}")

=== ARCHIVOS GUARDADOS ===
✅ Excel: ../Base_de_datos_limpia/Productos_limpio.xlsx
✅ CSV: ../Base_de_datos_limpia/Productos_limpio.csv
✅ Reporte: ../Base_de_datos_limpia/Reporte_Limpieza_Productos.txt

🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!
📁 Archivos disponibles en: ../Base_de_datos_limpia/


## 🎯 Conclusiones

### ✅ **Proceso Completado**
El análisis y limpieza de la tabla **Productos** se completó exitosamente. Los datos están ahora:

- **Limpios**: Sin espacios extra ni formatos inconsistentes
- **Estandarizados**: Formatos uniformes en nombres, categorías y precios
- **Normalizados**: Tipos de datos optimizados y columnas derivadas añadidas
- **Validados**: Verificación de integridad relacional con otras tablas

### 📊 **Archivos Generados**
Los resultados están guardados en la carpeta `Base_de_datos_limpia/`:
- `Productos_limpio.xlsx` - Datos limpios en formato Excel
- `Productos_limpio.csv` - Datos limpios en formato CSV
- `Reporte_Limpieza_Productos.txt` - Reporte detallado del proceso

### 🔄 **Próximos Pasos**
Los datos limpios de **Productos** están listos para:
- Análisis de ventas por producto y categoría
- Integración con las tablas **Detalle_ventas**, **Ventas** y **Clientes**
- Análisis de rentabilidad y rendimiento de productos
- Creación de dashboards y reportes de inventario

### 📝 **Características Especiales**
- **Categorización inteligente**: Conversión automática a categorías cuando es eficiente
- **Rangos de precio**: Clasificación automática en 5 niveles de precio
- **Validación de precios**: Detección de valores negativos o nulos
- **Optimización de memoria**: Tipos de datos optimizados para mejor rendimiento

In [39]:
#Muestra de los datos limpios cargados desde el archivo guardado.

Productos = "../Base_de_datos_limpia/Productos_limpio.xlsx"

p = pd.read_excel(Productos, index_col='id_producto')
p

,nombre_producto,categoria,precio_unitario,rango_precio,categoria_Alimentos,categoria_Limpieza
id_producto,,,,,,
1,Coca Cola 1.5L,Alimentos,2347,Medio,1,0
2,Pepsi 1.5L,Limpieza,4973,Muy Alto,0,1
3,Sprite 1.5L,Alimentos,4964,Muy Alto,1,0
4,Fanta Naranja 1.5L,Limpieza,2033,Bajo,0,1
5,Agua Mineral 500Ml,Alimentos,4777,Muy Alto,1,0
6,Jugo De Naranja 1L,Limpieza,4170,Muy Alto,0,1
7,Jugo De Manzana 1L,Alimentos,3269,Alto,1,0
8,Energética Nitro 500Ml,Limpieza,4218,Muy Alto,0,1
9,Yerba Mate Suave 1Kg,Alimentos,3878,Alto,1,0
